### Kaggle Problem

https://www.kaggle.com/c/santander-customer-transaction-prediction

### Setup

In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [41]:
import numpy as np
import pandas as pd
import zipfile
import os
from subprocess import check_output
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from __future__ import division

import datetime

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import re
import os

from sklearn import metrics

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import forest
from sklearn import metrics

import math

import IPython, graphviz, sklearn_pandas, sklearn, warnings

from matplotlib import pyplot as plt, rcParams, animation

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import PIL, os, numpy as np, math, collections, threading, json, bcolz, random, scipy, cv2

import gc

from sklearn import metrics

import xgboost as xgb

In [4]:
PATH = "data/"

In [5]:
!ls {PATH}

### Read data

In [1]:
#Training data
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/10385/298493/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1552187272&Signature=gisDRQ8RcsJjL8%2FmLp%2BIsosuToUxnqxADg5sc1J33uytCiVOYfIltK5GxPHa45XWCbBQB2C%2BV8rD8zSru2FpTxkajXWyokGM499%2FFIyeB5IpJ4flv3Ycf4d5wPzAi5eBSdAtewSXDMsR4qKRQKI3%2F9zKOI%2F%2FMZhhHpIwoDSpNszeI%2BX8X6m4pZQaS%2Fbj6i%2FOi5qaiPtvC4HRPPdcx%2BKDL%2FwGRstUcyGG5ac0y62eDfGNHOGCdhBXfBBgBEunEZ2DMpKSktXXIvx%2Furf98FsKJ6ZEWyrT%2FfYipM42j3v098GgSUFc%2BXFWNQadB1mraSHfXoVaPvxrYn3C73aadq%2F3lg%3D%3D" -O "data/train.csv.zip" -c
#Test data
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/10385/298493/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1552187241&Signature=LGbg4S87fjpS5RHtuXDtNQLGGsp29sLzLGycVhWe1Di9EgTsXtzMZ8Yj6tjnNtJXSchckTv4njyGP9x49yXuHtrZnUGAyY6YiBelFpkteYfiX93n53dS87r3R9kIaWiG%2BNQ85Tm5zkJH8gazepWA58%2Fe6rpHjoNa4R%2FnD5OJ33zwrukw0wAl4pVmB4iBmKES5vjMkVvH76Zh5BJfjYwUdU0gXFtaCJmxnGb6L6j1YSt%2BIbpw21vl5h2dio56aZWc%2FQUtinu9i1zNvMOx1N69VnO%2F6s%2BeflFls8kVD5fpirVXyQLgFtL6HiXOiLA2YyCw%2FNMfqH%2BzZ5hrS5SUdosipw%3D%3D" -O "data/test.csv.zip" -c
#Sample Sumbmission
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/10385/298493/sample_submission.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1552187196&Signature=QDaMp6tIDzk4xI%2Fv%2BWSolDzWPAoohTu4UpG7A2v8z6CLdsW4YYDKaosJfXzJOPHs25ufGXIX6c%2B9WSJq7voLjB5eTyHHyteGiDDNMQW4VS9p9iuLfV4aU2p87WGDjHBZgej5sRio2zPm4P2Pg9G3ID5l3BZ0U2MLWDcHVJnBpT4wh3zhUtn%2BFfiXYpxaj79ouma1XTaUWS0VQO%2B6p1%2BUya22P4319%2Bf9z2xM74m6jIDJuDk4ctZC5gl4PLJRT5BzBmbo71IWCKD2YUlSpFyew1alb5RjjurLU3V3pfWYoF1y4S6zQbqE%2B59Srz6WcB8OrP1AlVxfvt51T6WGoB%2Ft2Q%3D%3D" -O "data/sample_submission.csv.zip" -c

--2019-03-07 03:08:18--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/10385/298493/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1552187272&Signature=gisDRQ8RcsJjL8%2FmLp%2BIsosuToUxnqxADg5sc1J33uytCiVOYfIltK5GxPHa45XWCbBQB2C%2BV8rD8zSru2FpTxkajXWyokGM499%2FFIyeB5IpJ4flv3Ycf4d5wPzAi5eBSdAtewSXDMsR4qKRQKI3%2F9zKOI%2F%2FMZhhHpIwoDSpNszeI%2BX8X6m4pZQaS%2Fbj6i%2FOi5qaiPtvC4HRPPdcx%2BKDL%2FwGRstUcyGG5ac0y62eDfGNHOGCdhBXfBBgBEunEZ2DMpKSktXXIvx%2Furf98FsKJ6ZEWyrT%2FfYipM42j3v098GgSUFc%2BXFWNQadB1mraSHfXoVaPvxrYn3C73aadq%2F3lg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.240, 2607:f8b0:4004:814::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128240759 (122M) [application/zip]
Saving to: ‘data/train.csv.zip’

data/train.csv.zip  100%[===================>] 122.30M  76.5MB/s    in 1.6s    

2019-03-07 03

In [5]:
# Current directory
os.getcwd()

'/home/nbuser/DS_Explorations/Santander'

In [8]:
#Unzip files
dir_name = '/home/nbuser/DS_Explorations/Santander/data'
extension = ".zip"

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = dir_name + "/" + item
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [10]:
%%time
train_data = pd.read_csv('data/train.csv')#,nrows=1000)

CPU times: user 6.98 s, sys: 524 ms, total: 7.51 s
Wall time: 15 s


In [11]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:            15G        1.2G         10G        4.1M        3.5G         14G
Swap:            0B          0B          0B


In [13]:
#Just for display of DF on Notebook
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [14]:
display_all(train_data.head(5).transpose())

,0,1,2,3,4
ID_code,train_0,train_1,train_2,train_3,train_4
target,0,0,0,0,0
var_0,8.9255,11.5006,8.6093,11.0604,9.8369
var_1,-6.7863,-4.1473,-2.7457,-2.1518,-1.4834
var_2,11.9081,13.8588,12.0805,8.9522,12.8746
var_3,5.093,5.389,7.8928,7.1957,6.6375
var_4,11.4607,12.3622,10.5825,12.5846,12.2772
var_5,-9.2834,7.0433,-9.0837,-1.8361,2.4486
var_6,5.1187,5.6208,6.9427,5.8428,5.9405
var_7,18.6266,16.5338,14.6155,14.925,19.2514


In [15]:
#Test data
test_data = pd.read_csv('data/test.csv')#,1000)

In [16]:
#Save in feather format
train_data.to_feather('data/train_data')
test_data.to_feather('data/test_data')

In [28]:
#read from feather
train_data = pd.read_feather('data/train_data')
test_data = pd.read_feather('data/test_data')

/usr/local/lib/python3.6/dist-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


### Data Prep for modelling

In [29]:
train_data.shape

(200000, 202)

In [30]:
test_data.shape

(200000, 201)

In [31]:
display_all(train_data[['target']].isnull().sum()/len(train_data))

target    0.0
dtype: float64

In [18]:
#Columns different in train and test data
print("Variables not in test but in train : ", set(train_data.columns).difference(set(test_data.columns)))

Variables not in test but in train :  {'target'}


In [20]:
#get a sense of indvidual data types
display_all(train_data.dtypes)

ID_code     object
target       int64
var_0      float64
var_1      float64
var_2      float64
var_3      float64
var_4      float64
var_5      float64
var_6      float64
var_7      float64
var_8      float64
var_9      float64
var_10     float64
var_11     float64
var_12     float64
var_13     float64
var_14     float64
var_15     float64
var_16     float64
var_17     float64
var_18     float64
var_19     float64
var_20     float64
var_21     float64
var_22     float64
var_23     float64
var_24     float64
var_25     float64
var_26     float64
var_27     float64
var_28     float64
var_29     float64
var_30     float64
var_31     float64
var_32     float64
var_33     float64
var_34     float64
var_35     float64
var_36     float64
var_37     float64
var_38     float64
var_39     float64
var_40     float64
var_41     float64
var_42     float64
var_43     float64
var_44     float64
var_45     float64
var_46     float64
var_47     float64
var_48     float64
var_49     float64
var_50     f

In [33]:
def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.

    Parameters:
    -----------
    df: The data frame that will be changed.

    col: The column of data to fix by filling in missing data.

    name: The name of the new filled column in df.

    na_dict: A dictionary of values to create na's of and the value to insert. If
        name is not a key of na_dict the median will fill any missing data. Also
        if name is not a key of na_dict and there is no missing data in col, then
        no {name}_na column is not created.


    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col1'], 'col1', {})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1     2    2    True
    2     3    2   False


    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col2'], 'col2', {})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2


    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2

    >>> fix_missing(df, df['col1'], 'col1', {'col1' : 500})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1   500    2    True
    2     3    2   False
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            #df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

In [34]:
na_dict = {}
for n,c in train_data.items(): na_dict = fix_missing(train_data, c, n, na_dict)
    
for n,c in test_data.items(): na_dict = fix_missing(test_data, c, n, na_dict)

In [35]:
na_dict

{}

### Modelling

In [59]:
train_data.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [37]:
m = RandomForestClassifier(n_jobs=-1)
X = train_data.drop(['target','ID_code'],axis = 1)
Y = train_data.target
m.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [40]:
m.score(X,Y) #Accuracy

0.985225

In [53]:
from sklearn.metrics import accuracy_score, auc, classification_report, confusion_matrix, f1_score, log_loss, precision_recall_curve, roc_auc_score, roc_curve

pred = m.predict(X)
print('Accuracy_score: ', accuracy_score(pred, Y))
print('confusion_matrix: ', confusion_matrix(pred, Y))
print('Classification_Report: ', classification_report(pred, Y))

Accuracy_score:  0.985225
confusion_matrix:  [[179902   2955]
 [     0  17143]]
Classification_Report:               precision    recall  f1-score   support

          0       1.00      0.98      0.99    182857
          1       0.85      1.00      0.92     17143

avg / total       0.99      0.99      0.99    200000



In [60]:
metrics.roc_auc_score(Y, pred)

0.92648522241019

In [63]:
x_test = test_data.drop(['ID_code'],axis = 1)
y_test = m.predict(x_test)

In [64]:
submission = pd.DataFrame({"ID_code":test_data["ID_code"],"target":y_test})
submission.to_csv("submission_v0.csv", index=False)